In [6]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import pylab

Загружаем выборки.

In [7]:
data1 = pd.read_excel('Data_Extract_From_Gender_Statistics.xlsx', encoding = 'utf8')
def_data1 = pd.read_excel('Data_Extract_From_Gender_Statistics.xlsx', sheetname=1, encoding = 'utf8')

data2 = pd.read_excel('Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', encoding='utf8')
def_data2 = pd.read_excel('Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', sheetname=1, encoding='utf8')

data3 = pd.read_excel('Data_Extract_From_Millennium_Development_Goals.xlsx', encoding='utf8')
def_data3 = pd.read_excel('Data_Extract_From_Millennium_Development_Goals.xlsx', sheetname=1, encoding='utf8')

data_cnt = pd.read_csv('all.csv', encoding='utf8')

Очищаем данные от больших пропусков, точек и пр.

In [8]:
def Fillnan(data, years_c):
    for name in years_c:
        data[name] = data[name].apply(lambda x: np.nan if x==('..') else float(x))
    return data
    
def Dropempt(data, years_c):
    tmp=[]
    for ind, item in enumerate(data[years_c].as_matrix()):
        if np.nansum(item) == 0:
            tmp.append(ind)
    data = data.drop(data.index[tmp]).reset_index()
    return data.drop('index', axis=1)
    
def Clearing(data):
    years_c = [item for ind, item in enumerate(np.array(data.columns)) if ind not in (range(0,4))]
    if data['Country Code'].get_value(len(data)-5) == np.nan:
        data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    data = Dropempt(Fillnan(data, years_c), years_c)
    return data

Сортировка по регионам.

In [9]:
def Make_region(code):
    #print code
    if code in data_cnt['name'].values:
        #print code
        name = (data_cnt['name'][data_cnt['name'] == code].index.tolist())[0]
        return data_cnt['region'].get_value(name)
    else:
        return np.nan

def Sorting(data):
    if data['Country Name'].get_value(len(data)-5) == np.nan:
        data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    elif 'Region' not in data.columns:
        data['Region'] = data['Country Name'].apply(lambda x: Make_region(x))
        data['Region'].dropna()
        return data.reset_index()
    else:
        return data

In [10]:
data1 = Clearing(data1)
#заметил много шума.
del data1['2016 [YR2016]']

data2 = Clearing(data2)
del data2['2001 [YR2001]']

data3 = Clearing(data3)
del data3['2016 [YR2016]']

Эвристика
Преимущество данного подхода в том, что мы не потеряем единственные значения у некоторых стран. Но при этом, отразим актуальность, а значит и реальность.

In [11]:
def data_inc(data):
    magic = 0.08
    for ind, item in enumerate(np.array(data.columns)):
        if ind not in (range(0,4)):
        #попробую с интерполяцией
            data[item] = data[item].apply(lambda x: ((ind-2)*magic)*x if ((ind-2)*magic)>1 else x).interpolate()
    return data

def R_mean(data):
    if ('Mean' not in data.columns) and ('Region' not in data.columns):
        data['Mean']= data[[item for ind, item in enumerate(np.array(data.columns)) if ind not in (range(0,4))]].apply(lambda x: np.nanmean(x), axis = 1)
    elif ('Region' in data.columns):
        data['Mean']= data[[item for ind, item in enumerate(np.array(data.columns)) if ind not in [(range(0,4)), len(data.columns)-1]]].apply(lambda x: np.nanmean(x), axis = 1)
    return data

Чувствую себя отвратительно за такой код. Доделаю до конца и поправлю. 

In [12]:
data1 = Sorting(R_mean(data_inc(data1)))
data2 = Sorting(R_mean(data_inc(data2)))
data3 = Sorting(R_mean(data_inc(data3)))

Вот тут буду коррелировать, строить графики и пр

In [27]:
def podshape(t1,t2):
    cntrs1 = t1['Country Name'].tolist()
    cntrs2 = t2['Country Name'].tolist()
    x, y = [],[]
    if len(t1)!=len(t2):
        for contr in cntrs1:
            if contr in cntrs2:
                x.append(t1['Mean'][t1['Country Name']== contr].as_matrix()[0])
                y.append(t2['Mean'][t2['Country Name']== contr].as_matrix()[0])
        return x, y
    else:
        return t1['Mean'].as_matrix(), t2['Mean'].as_matrix()

def Shaping(t1,t2):
    if len(t1)==0 or len(t1)==0:
        print 'zero'
        return [0],[0]
    
    if len(t1)<len(t2):
        return podshape(t1,t2)
    elif len(t1)>len(t2):
        return podshape(t1,t2)
    else:
        return t1['Mean'].as_matrix(), t2['Mean'].as_matrix()
    
def Correlation(data):
    regions = data['Region'].unique()
    codes = data['Series Code'].dropna().unique()
    rez=pd.DataFrame()
    for region in regions:
        for code1 in codes:
            for code2 in codes:
                #print (region, code1, code2)
                Sh = Shaping(data[['Mean', 'Country Name']][data['Series Code']==code1][data['Region']==region], data[['Mean', 'Country Name']][data['Series Code']==code2][data['Region']==region])
                tmp = sts.pearsonr(Sh[0],Sh[1])
                rez = rez.append(pd.DataFrame(tmp[0], columns=[code1+':'+region], index=[code2+':'+region+':'+'cor-value']))
                rez = rez.append(pd.DataFrame(tmp[1], columns=[code1+':'+region], index=[code2+':'+region+':'+'p-value']))
                rez = rez.groupby(rez.index).first()
            #что-то да происходит.
            return rez
        print ('Passed: ' + region)
    return rez

In [28]:
Correlation(data1)

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,SH.HIV.ARTC.FE.ZS:Asia
"FP.CPI.TOTL.ZG:Asia,cor-value",-0.530157
"FP.CPI.TOTL.ZG:Asia,p-value",0.013431
"IC.FRM.FEMM.ZS:Asia,cor-value",0.086447
"IC.FRM.FEMM.ZS:Asia,p-value",0.709454
"NY.GDP.MKTP.CD:Asia,cor-value",0.050404
"NY.GDP.MKTP.CD:Asia,p-value",0.823725
"NY.GDP.MKTP.KD.ZG:Asia,cor-value",-0.270369
"NY.GDP.MKTP.KD.ZG:Asia,p-value",0.223625
"NY.GDP.PCAP.CD:Asia,cor-value",0.288693
"NY.GDP.PCAP.CD:Asia,p-value",0.192574
